In [1]:
#It is a good code for recorning the video For Real

In [2]:
%matplotlib inline
import numpy as np
import cv2
import random
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
import math

In [3]:
#Parameters
record = 1 # on=1 off=0
namerec = "sequence30-NTD10-K1-LSTM"
radius = 20  # Radius of the circle

In [4]:
import os

directory = r"F:\Courses\Paper\Mine\Paper3\Data"

# get a list of all .txt files in the directory
txt_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.txt')]

# print the number of .txt files found
print("Number of txt files:", len(txt_files))

# read the contents of each .txt file and store them in an array
Data = np.zeros((len(txt_files),50000,4))
for file_path in txt_files:
    for i in range(len(txt_files)):
        data = np.loadtxt(txt_files[i], dtype=object)
        data = [line.split(',') for line in data]

        # Convert the resulting list of lists to a NumPy array with the appropriate data types
        data = np.array(data, dtype=np.float32)
        #data = data.astype(np.int32)
        A = np.zeros((50000-data.shape[0], data.shape[1]))
        Data[i] = np.concatenate((data, A))


Number of txt files: 11


In [5]:
import pandas as pd

# Load the Excel file into a DataFrame
df = pd.read_excel(r'F:\Courses\Paper\Mine\Paper3\Animation\For recording\Properties-Part1.xlsx', sheet_name='Sheet1')

# Select the values from row 3 and onwards
Properties_Ani = df.values[1:]

In [6]:
# Load the Excel file into a DataFrame
df = pd.read_excel(r'F:\Courses\Paper\Mine\Paper3\Properties.xlsx', sheet_name='Sheet1')

# Select the values from row 3 and onwards
Properties_Uni = df.values[1:]

In [7]:
import numpy as np

# Assume the data array is already loaded into a variable named 'data'

# Find the indices where the values are "N" or "F"
idx_n = np.where(Properties_Ani == np.array(["N"], dtype=str))
idx_f = np.where(Properties_Ani == np.array(["F"], dtype=str))

# Replace "N" with "1"
Properties_Ani[idx_n] = 1

# Replace "F" with "2"
Properties_Ani[idx_f] = 2

# Find the indices where the values are "N" or "F"
idx_n = np.where(Properties_Uni == np.array(["N"], dtype=str))
idx_f = np.where(Properties_Uni == np.array(["F"], dtype=str))

# Replace "N" with "1"
Properties_Uni[idx_n] = 1

# Replace "F" with "2"
Properties_Uni[idx_f] = 2


In [8]:
Properties_Ani = np.array(Properties_Ani)
Properties_Ani = Properties_Ani.astype(float)
Properties_Uni = np.array(Properties_Uni)
Properties_Uni = Properties_Uni.astype(float)

In [9]:
resulotion = 1/30
Dim = round((Properties_Ani[-1,0]-Properties_Ani[0,0])/resulotion)
Properties_Ani_res = np.zeros((Dim, Properties_Ani.shape[1]))
t0 = 0
tf = 303
t = t0
Properties_Ani_inter = 0

for i in range(Dim):
    Properties_Ani_res[i] = Properties_Ani[Properties_Ani_inter]
    Properties_Ani_res[i,0] = t
    if t >= Properties_Ani[Properties_Ani_inter+1,0]:
        Properties_Ani_inter += 1
    
    t += resulotion

In [10]:
Dim = round((Properties_Uni[-1,0]-Properties_Uni[0,0])/resulotion)
Properties_Uni_res = np.zeros((Dim, Properties_Uni.shape[1]))
Data_res = np.zeros((len(txt_files), Dim, Data.shape[2]))
t0 = 5
tf = 620.8
t = t0
Data_inter = np.zeros(len(txt_files),dtype="int")
Properties_Uni_inter = 0
#for j in range(len(txt_files)):
for j in range(1):
    for i in range(Data[j].shape[0]):
        if Data[j,i,0] >= t0:
            Data_inter[j] = i - 1
            break
for i in range(Dim):
    Properties_Uni_res[i] = Properties_Uni[Properties_Uni_inter]
    Properties_Uni_res[i,0] = t
    if t >= Properties_Uni[Properties_Uni_inter+1,0]:
        Properties_Uni_inter += 1
    for j in range(len(txt_files)):
        if Data[j,Data_inter[j]+2,0]==0:
            continue
        Data_res[j,i] = Data[j, Data_inter[j]]
        Data_res[j,i,0] = t
        while True:
            if t >= Data[j, Data_inter[j] + 1,0]:
                Data_inter[j] += 1
            else:
                break
    
    t += resulotion

In [11]:
# Number 1
XBminN1 = 122 #X Body min Near
XBmaxN1 = 167 #X Body max Near

XBminF1 = 155 #X Body min Far
XBmaxF1 = 175 #X Body max Far


# Number 2
XBminN2 = 170 #X Body min Near
XBmaxN2 = 195 #X Body max Near

XBminF2 = 180 #X Body min Far
XBmaxF2 = 200 #X Body max Far


# Number 3
XBminN3 = 200 #X Body min Near
XBmaxN3 = 235 #X Body max Near

XBminF3 = 203 #X Body min Far
XBmaxF3 = 218 #X Body max Far


In [12]:
Model_data = np.zeros((Data_res.shape[0]*Data_res.shape[1],Properties_Uni_res.shape[1]))
Model_label = np.zeros((Data_res.shape[0]*Data_res.shape[1]), dtype="O")
inter = 0
for z in range(Data_res.shape[0]):
    for j in range(Data_res.shape[1]):
        if Data_res[z,j,0] == 0:
            continue
        Model_data[inter] = Properties_Uni_res[j]
        Model_label[inter] = "No Data"

        P = 0 # number of person 1
        if Properties_Uni_res[j,6*P+1] == 1: #if the person is present
            if Properties_Uni_res[j,6*P+2] == 1: #if the person is Near or Far
                if Properties_Uni_res[j,6*P+3] != 7: #if the person is stationary or Entering or Leaving
                    if XBminN1<Data_res[z,j,2]<XBmaxN1: #Body
                        Model_label[inter] = "Number 1"
                            
                elif Properties_Uni_res[j,6*P+3] == 7:  #if the person is stationary or moving
                    if 100<Data_res[z,j,2]<XBmaxN1: #Body
                        Model_label[inter] = "Number 1"

                        
            elif Properties_Uni_res[j,6*P+2] == 2: #if the person is Near or Far
                if Properties_Uni_res[j,6*P+3] != 7: #if the person is stationary or Entering or Leaving
                    if XBminF1<Data_res[z,j,2]<XBmaxF1: #Body
                        Model_label[inter] = "Number 1"

                elif Properties_Uni_res[j,6*P+3] == 7:  #if the person is stationary or moving
                    if 100<Data_res[z,j,2]<XBmaxN1: #Body
                        Model_label[inter] = "Number 1"


        P = 1 # number of person 2
        if Properties_Uni_res[j,6*P+1] == 1: #if the person is present
            if Properties_Uni_res[j,6*P+2] == 1: #if the person is Near or Far
                if Properties_Uni_res[j,6*P+3] != 7: #if the person is stationary or Entering or Leaving
                    if XBminN2<Data_res[z,j,2]<XBmaxN2: #Body
                        Model_label[inter] = "Number 2"
                            
                elif Properties_Uni_res[j,6*P+3] == 7:  #if the person is stationary or moving
                    if Properties_Uni_res[j,6*P+6] <=0:  #if the person is leaving or entering from left side
                        if 100<Data_res[z,j,2]<180: #Body
                            Model_label[inter] = "Number 2"
                    elif Properties_Uni_res[j,6*P+6] >=0:  #if the person is leaving or entering from left side
                        if 180<Data_res[z,j,2]<250: #Body
                            Model_label[inter] = "Number 2"
                            

            elif Properties_Uni_res[j,6*P+2] == 2: #if the person is Near or Far
                if Properties_Uni_res[j,6*P+3] != 7: #if the person is stationary or Entering or Leaving
                    if XBminF2<Data_res[z,j,2]<XBmaxF2: #Body
                        Model_label[inter] = "Number 2"

                            
                elif Properties_Uni_res[j,6*P+3] == 7:  #if the person is stationary or moving
                    if Properties_Uni_res[j,6*P+6] <=0:  #if the person is leaving or entering from left side
                        if 100<Data_res[z,j,2]<180: #Body
                            Model_label[inter] = "Number 2"
                    elif Properties_Uni_res[j,6*P+6] >=0:  #if the person is leaving or entering from left side
                        if 180<Data_res[z,j,2]<250: #Body
                            Model_label[inter] = "Number 2"
                            

        P = 2 # number of person 3
        if Properties_Uni_res[j,6*P+1] == 1: #if the person is present
            if Properties_Uni_res[j,6*P+2] == 1: #if the person is Near or Far
                if Properties_Uni_res[j,6*P+3] != 7: #if the person is stationary or Entering or Leaving
                    if XBminN3<Data_res[z,j,2]<XBmaxN3: #Body
                        Model_label[inter] = "Number 3"
                            
                elif Properties_Uni_res[j,6*P+3] == 7:  #if the person is stationary or moving
                    if XBminN3<Data_res[z,j,2]<250: #Body
                        Model_label[inter] = "Number 3"

            elif Properties_Uni_res[j,6*P+2] == 2: #if the person is Near or Far
                if Properties_Uni_res[j,6*P+3] != 7: #if the person is stationary or Entering or Leaving
                    if XBminF3<Data_res[z,j,2]<XBmaxF3: #Body
                        Model_label[inter] = "Number 3"
                        
                elif Properties_Uni_res[j,6*P+3] == 7:  #if the person is stationary or moving
                        if XBminF3<Data_res[z,j,2]<250: #Body
                            Model_label[inter] = "Number 3"
        inter +=1

In [13]:
Model_data = np.array(Model_data[0:inter])
Model_label = np.array(Model_label[0:inter])

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler = scaler.fit(Model_data)
Model_data_normal = scaler.transform(Model_data)  #scale the data

Model_data_normal = np.delete(Model_data_normal, 0, axis=1)

In [15]:
def Labell(String):
    if String == "Number 1":
        return 0
    elif String == "Number 2":
        return 1
    elif String == "Number 3":
        return 2

In [16]:
Persons_inter = np.zeros(len(txt_files))
inter = 0
for i in range(Model_data.shape[0]):
    if Model_data[i, 0] == 5:
        Persons_inter[inter] = i
        inter += 1

In [17]:
Persons_inter

array([     0.,  12133.,  30607.,  49081.,  67555.,  86029., 104503.,
       122977., 141451., 159925., 178399.])

In [18]:
mean = [ 3.06370221e+02,  8.22692802e-01,  1.22970646e+00,  3.93117390e+00,
  3.08086939e-01,  6.65301997e-02, -3.18157899e+01,  8.02654503e-01,
  1.19513595e+00,  3.76530555e+00,  2.83873360e-01,  1.62993402e-01,
 -3.84537240e-01,  8.15886384e-01,  1.22237178e+00,  3.61190209e+00,
  2.47362513e-01,  7.50483814e-02,  3.12343744e+01]
std = [176.46310571,   0.38192847,   0.72907873,   2.76399406,   0.46170269,
   0.24920661,  17.24564431 ,  0.39799529,   0.74279802,   2.72532234,
   0.45087612,   0.3935287,   10.19257654,   0.38757682,   0.73562885,
   2.63930238,   0.4314792,    0.29356123,  17.34610596]


In [19]:
Properties_Normal_res = np.array((Properties_Ani_res-mean)/std)
Properties_res = np.array(Properties_Ani_res[:,1:19])
Properties_Normal_res = np.array(Properties_Normal_res[:,1:19])

In [20]:
sequence = 30
Model_data_normal_sequence = np.zeros((Properties_Normal_res.shape[0],sequence,Properties_Normal_res.shape[1]),dtype='float')
inter = 0
for i in range(Model_data_normal_sequence.shape[0]-sequence):
    for j in range(sequence):
        Model_data_normal_sequence[inter,j] = Properties_Normal_res[i+j]
    inter += 1
        
Model_data_normal_sequence = np.array(Model_data_normal_sequence[0:inter])

In [21]:
import tensorflow as tf
from tensorflow import keras
import h5py

# Load the model from the HDF5 file
#loaded_model = keras.models.load_model(r'F:\Courses\Paper\Mine\Paper3\Results\LSTM-K3-TTP0.8-sequence30 (1).h5')
loaded_model1 = keras.models.load_model(r'F:\Courses\Paper\Mine\Paper3\Results new\Model\LSTM-K1-K_max10-sequence30-delta1.h5')
loaded_model2 = keras.models.load_model(r'F:\Courses\Paper\Mine\Paper3\Results new\Model\LSTM-K2-K_max10-sequence30-delta1.h5')
loaded_model3 = keras.models.load_model(r'F:\Courses\Paper\Mine\Paper3\Results new\Model\LSTM-K3-K_max10-sequence30-delta1.h5')
loaded_model4 = keras.models.load_model(r'F:\Courses\Paper\Mine\Paper3\Results new\Model\LSTM-K4-K_max10-sequence30-delta1.h5')
loaded_model5 = keras.models.load_model(r'F:\Courses\Paper\Mine\Paper3\Results new\Model\LSTM-K5-K_max10-sequence30-delta1.h5')
loaded_model6 = keras.models.load_model(r'F:\Courses\Paper\Mine\Paper3\Results new\Model\LSTM-K6-K_max10-sequence30-delta1.h5')
loaded_model7 = keras.models.load_model(r'F:\Courses\Paper\Mine\Paper3\Results new\Model\LSTM-K7-K_max10-sequence30-delta1.h5')
loaded_model8 = keras.models.load_model(r'F:\Courses\Paper\Mine\Paper3\Results new\Model\LSTM-K8-K_max10-sequence30-delta1.h5')
loaded_model9 = keras.models.load_model(r'F:\Courses\Paper\Mine\Paper3\Results new\Model\LSTM-K9-K_max10-sequence30-delta1.h5')
loaded_model10 = keras.models.load_model(r'F:\Courses\Paper\Mine\Paper3\Results new\Model\LSTM-K10-K_max10-sequence30-delta1.h5')


In [22]:
Model_data_normal_sequence.shape

(9060, 30, 18)

In [23]:
T = np.array([0, 60.87, 121.04, 187.07, 246.51])

In [24]:
y_pred1 = loaded_model1.predict(Model_data_normal_sequence[0:1826])
y_pred2 = loaded_model2.predict(Model_data_normal_sequence[1826:3631])
y_pred3 = loaded_model3.predict(Model_data_normal_sequence[3631:5612])
y_pred4 = loaded_model4.predict(Model_data_normal_sequence[5612:7395])
y_pred5 = loaded_model5.predict(Model_data_normal_sequence[7395:9060])


53/53 [==============================] - 2s 11ms/step


In [25]:
y_pred= np.concatenate((y_pred1, y_pred2, y_pred3, y_pred4, y_pred5))
y_pred.shape

(9060, 3)

In [29]:
mean = np.array([ 0.82279619,   1.22980571,   3.9329219,    0.30791619,   0.06652952,
                    -31.8272,      0.80288,  1.19558095,    3.76673524,   0.28387048,
                  0.16338286,  -0.37188571,  0.81583238,    1.22253714,   3.61266286,
                  0.24752762,   0.07504762,  31.2256    ])

STD = np.array([0.38184109,  0.72897371,  2.7642141,   0.46163168,  0.24920543, 17.25409012,
                0.39782371,  0.74267694,  2.72513013,  0.45087474,  0.39428699, 10.23517679,
                0.38762083,  0.73576464,  2.64014798,  0.43157583,  0.29355984, 17.34107153])

def Normalize(Input, Mean, STD):
    Input_normalized = (np.array(Input)-np.array(mean))/np.array(STD)
    return Input_normalized

In [31]:
mylist = ['Number 1',
          'Number 2',
          'Number 3']

In [104]:
XX_test = np.zeros((1,30,18))
for i in range(30):
    A = np.array([[0, 0, 0, 0, 0, 0], #Stand #Near #Activity #Talking #pointed #angle
                [1, 2, 1, 0, 0, 0],
                [1, 1, 4, 1, 0, 60]])
    A = A.reshape(1,18)
    A = Normalize(A, mean, STD)
    XX_test[0,i] = A

a = loaded_model2.predict(XX_test)
b = a.argmax()
print(mylist[b])
print(a[0,b])

a[0,b] = 0
b = a.argmax()
print(mylist[b])
print(a[0,b])

#a[0,b] = 0
#b = a.argmax()
#print(mylist[b])
#print(a[0,b])

1/1 [==============================] - 0s 63ms/step
Number 3
0.9629633
Number 2
0.036983952


In [27]:
Model_data_normal_sequence[0,-1]

array([ 0.4642419 , -0.315064  ,  1.11028679, -0.66728426, -0.26696804,
       -2.50406476,  0.49584883,  1.08355708,  1.18690344, -0.62960389,
       -0.41418428,  5.92436436, -2.10509592, -1.66166917, -1.36850636,
       -0.57328954, -0.25564814, -1.80065627])

In [28]:
y_pred[0]

array([9.995666e-01, 3.471760e-04, 8.610123e-05], dtype=float32)

In [29]:
#y_pred = model.predict(X_train)
#y_pred = loaded_model.predict(Model_data_normal_sequence)
Label = np.zeros(y_pred.shape[0],dtype='int') 
for i in range(Label.shape[0]):
    a=y_pred[i]
    a=a.argmax()
    if Model_data_normal_sequence[250*30,-1,6*a]==0:
        y_pred[i,a]=0
        a=y_pred[i]
        a=a.argmax()
    Label[i] = a 

In [30]:
mylist = ['Number 1',
          'Number 2',
          'Number 3']

In [31]:
Properties_res[100]

array([  1.,   1.,   7.,   0.,   0., -45.,   1.,   2.,   7.,   0.,   0.,
        30.,   0.,   0.,   0.,   0.,   0.,   0.])

In [32]:
def Participants_look(Object,i):
    XX = 0
    YY = 0

    if Object =='Number 1':
        if Properties_res[i,1]==1:
            XX = int(5.5*Properties_res[i,5]+512.5)
            YY = int(415)

        else:
            XX = int(10.66*Properties_res[i,5]+740)
            YY = int(435)
           
            
    elif Object =='Number 2':
        if Properties_res[i,7]==1:
            XX = int(11.06*Properties_res[i,11]+930)
            YY = int(415)
        else:                       
            XX = int(14.66*Properties_res[i,11]+940)
            YY = int(435)
                
                    
    elif Object =='Number 3':
        if Properties_res[i,12]==1:
            XX = int(7.33*Properties_res[i,17]+1270)
            YY = int(415)
                
        else:
            XX = int(10.66*Properties_res[i,17]+1180)
            YY = int(435)

    return XX,YY
            

In [33]:
from scipy.stats import multivariate_normal

def Star(Object,i):
    max_rep = 0
    multipy = 100  #50
    power = 2 #2
    XX = 0
    YY = 0

    if Object =='Number 1':
        if Properties_res[i,1]==1:
            if Properties_res[i,2]!=7 and Properties_res[i,2]!=2:
                max_rep = 0
                #max_rep = 6
                #XX = int(5.5*Properties_res[i,5]+512.5)
                #YY = int(415)
                Mean1N = np.array([265, 415])
                cov1N = np.array([[440, 73],[73, 730]])
                while max_rep<5:
                    x, y = np.random.multivariate_normal(Mean1N, cov1N, 1).T
                    var = multivariate_normal(mean=Mean1N, cov=cov1N)
                    max_rep = round((var.pdf(np.array([x[0],y[0]]))/var.pdf(Mean1N))**power*multipy)
                    XX = round(x[0])
                    YY = round(y[0])
            else:
                XX = int(5.5*Properties_res[i,5]+512.5)
                YY = int(415)

        else:
            if Properties_res[i,2]!=7 and Properties_res[i,2]!=2:
                max_rep = 0
                Mean1F = np.array([420, 435])
                cov1F = np.array([[232, 28],[28, 534]])
                while max_rep<5:
                    x, y = np.random.multivariate_normal(Mean1F, cov1F, 1).T
                    var = multivariate_normal(mean=Mean1F, cov=cov1F)
                    max_rep = round((var.pdf(np.array([x[0],y[0]]))/var.pdf(Mean1F))**power*multipy)
                    XX = round(x[0])
                    YY = round(y[0])
            else:
                XX = int(10.66*Properties_res[i,5]+740)
                YY = int(435)
           
            
    elif Object =='Number 2':
        if Properties_res[i,7]==1:
            if Properties_res[i,8]!=7 and Properties_res[i,8]!=2:
                max_rep = 0
                Mean2N = np.array([930, 415])
                cov2N = np.array([[442, 34],[34, 447]])
                while max_rep<5:
                    x, y = np.random.multivariate_normal(Mean2N, cov2N, 1).T
                    var = multivariate_normal(mean=Mean2N, cov=cov2N)
                    max_rep = round((var.pdf(np.array([x[0],y[0]]))/var.pdf(Mean2N))**power*multipy)
                    XX = round(x[0])
                    YY = round(y[0])
            else:
                XX = int(11.06*Properties_res[i,11]+930)
                YY = int(415)
        else:                       
            if Properties_res[i,8]!=7 and Properties_res[i,8]!=2:
                max_rep = 0
                Mean2F = np.array([940, 435])
                cov2F = np.array([[216, 48],[48, 464]])
                while max_rep<5:
                    x, y = np.random.multivariate_normal(Mean2F, cov2F, 1).T
                    var = multivariate_normal(mean=Mean2F, cov=cov2F)
                    max_rep = round((var.pdf(np.array([x[0],y[0]]))/var.pdf(Mean2F))**power*multipy)
                    XX = round(x[0])
                    YY = round(y[0])
            else:
                XX = int(14.66*Properties_res[i,11]+940)
                YY = int(435)
                
                    
    elif Object =='Number 3':
        if Properties_res[i,12]==1:
            if Properties_res[i,13]!=7 and Properties_res[i,13]!=2:
                max_rep = 0
                Mean3N = np.array([1600, 415])
                cov3N = np.array([[440, 73],[73, 730]])
                while max_rep<5:
                    x, y = np.random.multivariate_normal(Mean3N, cov3N, 1).T
                    var = multivariate_normal(mean=Mean3N, cov=cov3N)
                    max_rep = round((var.pdf(np.array([x[0],y[0]]))/var.pdf(Mean3N))**power*multipy)
                    XX = round(x[0])
                    YY = round(y[0])
                
            else:
                XX = int(7.33*Properties_res[i,17]+1270)
                YY = int(415)
                
        else:
            if Properties_res[i,12]!=7 and Properties_res[i,12]!=2:
                max_rep = 0
                Mean3F = np.array([1500, 435])
                cov3F = np.array([[232, 28],[28, 534]])
                while max_rep<5:
                    x, y = np.random.multivariate_normal(Mean3F, cov3F, 1).T
                    var = multivariate_normal(mean=Mean3F, cov=cov3F)
                    max_rep = round((var.pdf(np.array([x[0],y[0]]))/var.pdf(Mean3F))**power*multipy)
                    XX = round(x[0])
                    YY = round(y[0])
            else:
                XX = int(10.66*Properties_res[i,17]+1180)
                YY = int(435)

    return XX,YY, max_rep
            
            

In [34]:
cap = cv2.VideoCapture(r'F:\Courses\Paper\Mine\Paper3\Animation\For recording\Animation-Part 1.mkv')

record = 1
delta = 1

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) + 0.5)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) + 0.5)
size = (width, height)
if record == 1:
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(r"F:\Courses\Paper\Mine\Paper3/"+namerec+".avi", fourcc, 30.0, size)

    
#Parameters    
i = 0
w = 0
counter = 0
max_rep = 0
str_count = "Hi"
XX_old = 1920/2
YY_old = 900

while(cap.isOpened()):
    
    #For python
    ret, frame = cap.read()
    if ret == False or i==30*5*60:
        break
        
        
    
        
    cv2.namedWindow("frame", cv2.WINDOW_NORMAL)        # Create window with freedom of dimensions
    cv2.resizeWindow("frame", 2000, 800)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    #For time
    cv2.putText(frame,"t="+str(round(i/30,2))+"s", (100,100), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2)
    
    #Participants
    #for j in Persons_inter:
    #    Look_X, Look_Y = Participants_look(Model_label[int(j)+i],i)
    #    cv2.putText(frame, "*", (Look_X, Look_Y), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),2) #Detecter Model
        
    for j in range(Persons_inter.shape[0]):
        angle = j * (2 * math.pi / Persons_inter.shape[0])
        x =  radius * math.cos(angle)
        y =  radius * math.sin(angle)
        Look_X, Look_Y = Participants_look(Model_label[int(Persons_inter[j])+i],i)
        if Look_X>0:
            cv2.putText(frame, "*", (Look_X+int(x), Look_Y+int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),2) #Detecter Model
            


    #For detect
    if i >= sequence:
        #Update after delta
        if i%delta == 0: 
            A = Label[w]
            w = w + 1

        if str_count != mylist[A] or counter >= max_rep:
            str_count = mylist[A]
            XX, YY, max_rep = Star(mylist[A],i) 
            counter = 0
        else:
            counter = counter + 1

    else:
        XX = int(XX_old)
        YY = int(YY_old)

    XX_draw = int(XX_old + (XX - XX_old)/10)
    YY_draw = int(YY_old + (YY - YY_old)/10)

    XX_old = XX_draw
    YY_old = YY_draw


    cv2.putText(frame, "*", (XX_draw, YY_draw), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2) #Detecter Model


    if record==1:
        out.write(frame)
        
    if i%30==0:
        Name_photo = "Paper3-Photo"+str(i)
        cv2.imwrite(r"F:\Courses\Paper\Mine\Paper3\Snapshots/"+Name_photo+'.jpg', frame)

    cv2.imshow('frame', frame)
    
    i += 1
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    cv2.waitKey(1)
    
    
cap.release()
cv2.destroyAllWindows()

In [ ]:
cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
# for i in Label:
#     print(i)

In [ ]:
Label[0]

In [ ]:
Star("Number 1",31)

In [ ]:
Properties_res[31]

In [ ]:
Mean1N = np.array([265, 415])
cov1N = np.array([[440, 34],[34, 730]])
x, y = np.random.multivariate_normal(Mean1N, cov1N, 1).T
var = multivariate_normal(mean=Mean1N, cov=cov1N)
max_rep = round((var.pdf(np.array([x[0],y[0]]))/var.pdf(Mean1N))**power*multipy)

In [ ]:
x

In [ ]:
Properties_res[i]

In [ ]:
i

In [ ]:
Label[250*30]

In [ ]:
y_pred[250*30]

In [ ]:
Model_data_normal_sequence[250*30,-1]

In [ ]:
if Model_data_normal_sequence[250*30,-1]

In [ ]:
a=y_pred[250*30]
a=a.argmax()
if Model_data_normal_sequence[250*30,-1,6*a]==0:
    y_pred[250*30,a]=0
    a=y_pred[250*30]
    a=a.argmax()

In [ ]:
Model_data_normal_sequence[250*30,-1,6*a]==0

In [ ]:
for j in Persons_inter:
    print(int(j))
    #Look_X, Look_Y = Participants_look(Model_label[j+i],i)
    #cv2.putText(frame, "*", (Look_X, Look_Y), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),2) #Detecter Model

In [ ]:
import math

radius = 30  # Radius of the circle
num_stars = 11  # Number of stars

for i in range(num_stars):
    angle = i * (2 * math.pi / num_stars)
    x =  radius * math.cos(angle)
    y =  radius * math.sin(angle)
    print(x,y)

In [ ]:
x

In [ ]:
y

In [ ]:
61%30